# Atelier Pokémon – Partie 1 

Ce notebook charge et prépare les données (Pokedex + combats), corrige `LEGENDAIRE`, agrège les statistiques de combats, et sauvegarde un `dataset.tsv` pour la Partie 3.

In [ ]:
import pandas as pd


pd.set_option('display.max_columns', None)
pd.set_option('display.width', 140)
pd.set_option('future.no_silent_downcasting', True)

print("Pandas version:", pd.__version__)

## Chargement du Pokédex

In [ ]:

nosPokemons = pd.read_csv("datas/pokedex.csv")
print("Colonnes du Pokédex:")
print(nosPokemons.columns.values)
nosPokemons.head(10)

## Transformation de `LEGENDAIRE` (VRAI/FAUX → 1/0)

In [ ]:
nosPokemonsClean = nosPokemons.copy()
nosPokemonsClean["LEGENDAIRE"] = (
    nosPokemonsClean["LEGENDAIRE"]
    .replace({"VRAI": 1, "FAUX": 0})
    .astype("int32")
)
nosPokemonsClean.head(5)

## Vérifications de base

In [ ]:
print("Shape:", nosPokemonsClean.shape)
nosPokemonsClean.info()

## Recherche des valeurs manquantes (NOM manquant)

In [ ]:
missing_name = nosPokemonsClean[nosPokemonsClean['NOM'].isnull()]
missing_name

In [ ]:

if not missing_name.empty:
    idx = missing_name.index[0]
    nosPokemonsClean.loc[idx, 'NOM'] = "Colossinge"
    print("Nom corrigé à l'index", idx, "->", nosPokemonsClean.loc[idx, 'NOM'])

## Chargement des combats

In [ ]:

combats = pd.read_csv("datas/combats.csv")
print("Colonnes combats:", combats.columns.values)
print("Shape combats:", combats.shape)
combats.head(10)

## Comptage des apparitions (1ère et 2e position)

In [ ]:
nbPrem = combats.groupby('Premier_Pokemon').size().rename("count")
nbSec  = combats.groupby('Second_Pokemon').size().rename("count")

nombreTotalDeCombats = nbPrem.add(nbSec, fill_value=0).astype(int)
nombreTotalDeCombats.head()

## Nombre de victoires par Pokémon

In [ ]:
nombreDeVictoires = combats.groupby('Pokemon_Gagnant').size().rename("NBR_VICTOIRES").astype(int)
nombreDeVictoires.head()

## Agrégation : NBR_COMBATS, NBR_VICTOIRES, % de victoires

In [ ]:
listeAAgreger = nombreDeVictoires.to_frame()
listeAAgreger['NBR_COMBATS'] = nombreTotalDeCombats.reindex(listeAAgreger.index).fillna(0).astype(int)

listeAAgreger['POURCENTAGE_DE_VICTOIRES'] = (listeAAgreger['NBR_VICTOIRES'] / listeAAgreger['NBR_COMBATS'].replace(0, pd.NA)).astype(float)
listeAAgreger = listeAAgreger.sort_index()
listeAAgreger.head(10)

## Fusion avec le Pokédex → `nouveauPokedex`

In [ ]:
nouveauPokedex = nosPokemonsClean.merge(listeAAgreger, left_on='NUMERO', right_index=True, how='left')
nouveauPokedex.head(10)

## Aperçu statistique & sauvegarde du dataset (Partie 3)

In [ ]:
display(nouveauPokedex.describe(include='all'))


import os
os.makedirs("datas", exist_ok=True)
out_path = "datas/dataset.csv"  
nouveauPokedex.to_csv(out_path, sep="\t", index=False)
print("Dataset sauvegardé ->", out_path)